In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.options.display.max_rows = 50
pd.options.display.max_columns = None
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Twitter
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False
rc('font', family='AppleGothic')
import seaborn as sns
import math
from collections import Counter

In [2]:
# # -*- coding: utf-8 -*-
# import pandas as pd
# %matplotlib inline
# from collections import Counter
# import math
# import matplotlib.pyplot as plt
# from matplotlib import rc
# import seaborn as sns
# plt.rcParams['axes.unicode_minus'] = False
# rc('font', family='AppleGothic')
# import nltk
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
# import numpy as np

## Library

In [3]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
mecab = Mecab()
twitter = Twitter()

/Users/bagjongsu/anaconda3/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
libraries = {'hannanum' : hannanum, 'kkma' : kkma, 'mecab' : mecab, 'komoran' : komoran, 'twitter' : twitter}

## Input Data

In [5]:
df = pd.read_excel('./PLAZA_INTENT.xlsx')
print(df.shape)
df.head(10)

(4212, 2)


,input,intent
0,신었던 신발을 교환하고 싶어요,교환_신발
1,착화한 신발 환불 가능할까요?,교환_신발
2,신발 교환 방법,교환_신발
3,교환 신발,교환_신발
4,신발 한번밖에 안 신었어요,교환_신발
5,신발도 교환 되나요?,교환_신발
6,신발을 구입 후 1회 착화하였는데 발이 너무 아파 신을 수가 없는데 교환 가능한지요?,교환_신발
7,신발이 안맞아서 못신겠어요,교환_신발
8,신었던 신발 교환 가능한가요?,교환_신발
9,신었던 신발 바꿀수 있나요?,교환_신발


## Result

### 1 - 불용어 포함

In [41]:
import time
inputs = df['input'].values.tolist()
arr_list = []
i = 0
result_df = pd.DataFrame({})

for key, value in libraries.items():
    start_time = time.time()
    arr_ = []
    print('library : ', key)
    try:
        for input_ in inputs:
            i += 1
            if type(input_) != str:
                if math.isnan(input_):
                    continue
            # 한 input에서 중복되는 단어는 한 개로 취급
            arr_ += list(set(value.nouns(input_)))
            arr_list.append(arr_)
    except:
        print(i, input_, type(input_), math.isnan(input_))
    
    cnt = Counter(arr_)
    result_df[key] = cnt.most_common()[:30]
    print(cnt.most_common()[:30])
    print(f"{len(inputs)}개, 수행시간 : {time.time() - start_time}초")
    print()

library :  hannanum
[('상품권', 284), ('수', 251), ('어디', 234), ('가능', 124), ('리', 124), ('문화센터', 123), ('마', 118), ('백화점', 99), ('구매', 95), ('오늘', 95), ('매장', 94), ('언제', 91), ('교환', 81), ('방법', 78), ('상품', 77), ('수원점', 75), ('환불', 73), ('카드', 69), ('취소', 65), ('문화아카데미', 64), ('쿠폰', 63), ('강좌', 62), ('뭐', 61), ('영업', 59), ('멤버스', 55), ('사용', 55), ('적립', 54), ('분', 54), ('점', 54), ('휴무', 53)]
4212개, 수행시간 : 4.555830001831055초

library :  kkma
[('상품권', 313), ('어디', 283), ('수', 254), ('가능', 233), ('카드', 212), ('문화', 207), ('점', 180), ('가요', 167), ('매장', 167), ('방법', 163), ('주차', 163), ('센터', 160), ('쿠폰', 159), ('마일리지', 158), ('버스', 157), ('월', 146), ('백화점', 142), ('구매', 140), ('수원', 140), ('문화센터', 123), ('회원', 120), ('하나', 118), ('환불', 112), ('휴무', 112), ('상품', 108), ('가입', 107), ('오늘', 102), ('언제', 98), ('신청', 96), ('시간', 92)]
4212개, 수행시간 : 78.78860712051392초

library :  mecab
[('상품권', 313), ('어디', 286), ('점', 264), ('수', 258), ('가능', 237), ('문화', 207), ('카드', 203), ('버스', 161), ('멤', 160), 

### 2 - 불용어 제외

In [17]:
stopwords = pd.read_csv('./stopwords-ko.txt', sep='\\')
stopwords_set = set(stopwords.value.values)
print('불용어 개수 :', len(stopwords_set))
stopwords.head(10)

불용어 개수 : 595


,value
0,가
1,가까스로
2,가령
3,각
4,각각
5,각자
6,각종
7,갖고말하자면
8,같다
9,같이


In [42]:
import time
inputs = df['input'].values.tolist()
arr_list = []
i = 0
result_df_stop = pd.DataFrame({})

for key, value in libraries.items():
    start_time = time.time()
    arr_ = []
    print('library : ', key)
    try:
        for input_ in inputs:
            i += 1
            if type(input_) != str:
                if math.isnan(input_):
                    continue
            # 한 input에서 중복되는 단어는 한 개로 취급
            arr_ += list(set(value.nouns(input_)))
            arr_list.append(arr_)
    except:
        print(i, input_, type(input_), math.isnan(input_))
    
    cnt = Counter(arr_)
    result_df_stop[key + '_stop'] = cnt.most_common()[:30]
    print(cnt.most_common()[:30])
    print(f"{len(inputs)}개, 수행시간 : {time.time() - start_time}초")
    print()

library :  hannanum
[('상품권', 284), ('수', 251), ('어디', 234), ('가능', 124), ('리', 124), ('문화센터', 123), ('마', 118), ('백화점', 99), ('구매', 95), ('오늘', 95), ('매장', 94), ('언제', 91), ('교환', 81), ('방법', 78), ('상품', 77), ('수원점', 75), ('환불', 73), ('카드', 69), ('취소', 65), ('문화아카데미', 64), ('쿠폰', 63), ('강좌', 62), ('뭐', 61), ('영업', 59), ('멤버스', 55), ('사용', 55), ('적립', 54), ('분', 54), ('점', 54), ('휴무', 53)]
4212개, 수행시간 : 4.019683122634888초

library :  kkma
[('상품권', 313), ('어디', 283), ('수', 254), ('가능', 233), ('카드', 212), ('문화', 207), ('점', 180), ('가요', 167), ('매장', 167), ('방법', 163), ('주차', 163), ('센터', 160), ('쿠폰', 159), ('마일리지', 158), ('버스', 157), ('월', 146), ('백화점', 142), ('구매', 140), ('수원', 140), ('문화센터', 123), ('회원', 120), ('하나', 118), ('환불', 112), ('휴무', 112), ('상품', 108), ('가입', 107), ('오늘', 102), ('언제', 98), ('신청', 96), ('시간', 92)]
4212개, 수행시간 : 76.61243605613708초

library :  mecab
[('상품권', 313), ('어디', 286), ('점', 264), ('수', 258), ('가능', 237), ('문화', 207), ('카드', 203), ('버스', 161), ('멤', 160), 

# Data

In [48]:
print(df.shape)
df.head(10)

(4212, 2)


,input,intent
0,신었던 신발을 교환하고 싶어요,교환_신발
1,착화한 신발 환불 가능할까요?,교환_신발
2,신발 교환 방법,교환_신발
3,교환 신발,교환_신발
4,신발 한번밖에 안 신었어요,교환_신발
5,신발도 교환 되나요?,교환_신발
6,신발을 구입 후 1회 착화하였는데 발이 너무 아파 신을 수가 없는데 교환 가능한지요?,교환_신발
7,신발이 안맞아서 못신겠어요,교환_신발
8,신었던 신발 교환 가능한가요?,교환_신발
9,신었던 신발 바꿀수 있나요?,교환_신발


# 라이브러리 속도 비교

* hannanum : 약 4초
    
* kkma : 약 75초
    
* mecab : 약 0.1초
    
* komoran : 약 2초
    
* twitter : 약 6초

# 불용어 모음집

In [50]:
stopwords.tail(10)

,value
585,형식으로 쓰여
586,혹시
587,혹은
588,혼자
589,훨씬
590,휘익
591,휴
592,흐흐
593,흥
594,힘입어


In [44]:
df_concat = pd.concat([result_df, result_df_stop], axis=1)
df_concat

,hannanum,kkma,mecab,komoran,twitter,hannanum_stop,kkma_stop,mecab_stop,komoran_stop,twitter_stop
0,"(상품권, 284)","(상품권, 313)","(상품권, 313)","(수, 357)","(상품권, 312)","(상품권, 284)","(상품권, 313)","(상품권, 313)","(수, 357)","(상품권, 312)"
1,"(수, 251)","(어디, 283)","(어디, 286)","(상품권, 313)","(하나요, 214)","(수, 251)","(어디, 283)","(어디, 286)","(상품권, 313)","(하나요, 214)"
2,"(어디, 234)","(수, 254)","(점, 264)","(점, 221)","(카드, 190)","(어디, 234)","(수, 254)","(점, 264)","(점, 221)","(카드, 190)"
3,"(가능, 124)","(가능, 233)","(수, 258)","(문화, 207)","(어디, 167)","(가능, 124)","(가능, 233)","(수, 258)","(문화, 207)","(어디, 167)"
4,"(리, 124)","(카드, 212)","(가능, 237)","(휴, 206)","(매장, 165)","(리, 124)","(카드, 212)","(가능, 237)","(휴, 206)","(매장, 165)"
5,"(문화센터, 123)","(문화, 207)","(문화, 207)","(방법, 163)","(방법, 163)","(문화센터, 123)","(문화, 207)","(문화, 207)","(방법, 163)","(방법, 163)"
6,"(마, 118)","(점, 180)","(카드, 203)","(매장, 162)","(수, 161)","(마, 118)","(점, 180)","(카드, 203)","(매장, 162)","(수, 161)"
7,"(백화점, 99)","(가요, 167)","(버스, 161)","(주차, 160)","(쿠폰, 157)","(백화점, 99)","(가요, 167)","(버스, 161)","(주차, 160)","(쿠폰, 157)"
8,"(구매, 95)","(매장, 167)","(멤, 160)","(센터, 160)","(버스, 152)","(구매, 95)","(매장, 167)","(멤, 160)","(센터, 160)","(버스, 152)"
9,"(오늘, 95)","(방법, 163)","(마일리지, 160)","(쿠폰, 157)","(휴무, 151)","(오늘, 95)","(방법, 163)","(마일리지, 160)","(쿠폰, 157)","(휴무, 151)"


# 이슈 사항

* 현재 python 및 python library를 활용해 작업하고 있는데, java spring 프레임워크에서 그대로 활용 가능한지 여부

  => 인터넷을 통해 알아본 결과 가능하다고 함. 정확하게 확인 필요
  
  => 불용어 모음집 추가
  
  => 불용어 모음집에 단어 추가할 수 있는 기능
  
  => DB 적재위한 컬럼 정의(sprint2에서 하자)
  
  => 서버 올려두기
  
  => 엑셀 스크럼 작업량 load

# 작업 계획

* 일괄 테스트 기능(예제 샘플이 담긴 파일 업로드 시, 각 예제에 해당하는 출력 결과를 파일 형태로 반환) skill_id 단위 작업